## Логистическая регрессия

[Ссылка на полную статью](https://habr.com/ru/company/ods/blog/323890/)

In [1]:
import pandas as pd
import numpy as np

In [2]:
sessions = pd.read_csv('../data/alice/train_sessions.csv', index_col='session_id')

In [3]:
sessions.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [4]:
times = ['time%s' % i for i in range(1,11)]
sites = ['site%s' % i for i in range(1,11)]
sessions[times] = sessions[times].apply(pd.to_datetime)
sessions[sites] = sessions[sites].fillna(0).astype('int')
sessions.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,0,NaT,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
2,890,2014-02-22 11:19:50,941,2014-02-22 11:19:50,3847,2014-02-22 11:19:51,941,2014-02-22 11:19:51,942,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847,2014-02-22 11:19:52,3846,2014-02-22 11:19:52,1516,2014-02-22 11:20:15,1518,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39,2013-12-16 16:40:18,14768,2013-12-16 16:40:19,14769,2013-12-16 16:40:19,37,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768,2013-12-16 16:40:20,14768,2013-12-16 16:40:21,14768,2013-12-16 16:40:22,14768,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782,2014-03-28 10:52:42,782,2014-03-28 10:53:12,782,2014-03-28 10:53:42,782,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782,2014-03-28 10:55:12,782,2014-03-28 10:55:42,782,2014-03-28 10:56:12,782,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177,2014-02-28 10:55:22,175,2014-02-28 10:55:22,178,2014-02-28 10:55:23,177,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175,2014-02-28 10:55:59,177,2014-02-28 10:55:59,177,2014-02-28 10:57:06,178,2014-02-28 10:57:11,0


In [5]:
df_sites = sessions[sites]
df_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178


In [6]:
from scipy.sparse import csr_matrix

In [7]:
csr_matrix?

Init signature: csr_matrix(arg1, shape=None, dtype=None, copy=False)
Docstring:     
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSR representation where the column indices for
        row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
        corresponding values are stored in ``data[indptr[i]:indptr[i+1]]``.
        If the shape parameter is not supplied, the matrix dimensions
        are inferred 

In [8]:
sites_flatten = df_sites.values.flatten()
sites_flatten.shape[0]

2535610

In [9]:
sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0,sites_flatten.shape[0]+10,10),
    )
)[:,1:]

In [10]:
sites_sparse.shape

(253561, 41601)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = sessions['target']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(sites_sparse, y, random_state=10, train_size = 0.7)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(177492, 41601) (76069, 41601) (177492,) (76069,)


In [13]:
logit = LogisticRegression(C=1.0,n_jobs=-1, random_state=10)
logit.fit(X_train, y_train)

LogisticRegression(n_jobs=-1, random_state=10)

In [14]:
y_predict = logit.predict(X_test)

In [15]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_predict)

0.5863483389482806

In [16]:
def get_roc (X, y, C=1.0, ratio = 0.7, random_state = 10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, train_size = ratio)
    logit = LogisticRegression(C=C,n_jobs=-1, random_state=random_state)
    y_predict = logit.fit(X_train, y_train).predict(X_test)
    return roc_auc_score(y_test, y_predict)

In [17]:
get_roc(sites_sparse, y)

0.5863483389482806

In [18]:
first_time=sessions[times[0]]
first_time

session_id
1        2014-02-20 10:02:45
2        2014-02-22 11:19:50
3        2013-12-16 16:40:17
4        2014-03-28 10:52:12
5        2014-02-28 10:53:05
                 ...        
253557   2013-11-25 10:26:54
253558   2013-03-12 16:01:15
253559   2013-09-12 14:05:03
253560   2013-12-19 15:20:22
253561   2014-04-25 09:56:52
Name: time1, Length: 253561, dtype: datetime64[ns]

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [20]:
# Признак даты первого входа в сессии
sessions['year_month']=first_time.apply(lambda x: x.year * 100 + x.month)
sessions['year_month_scaled'] = scaler.fit_transform(sessions['year_month'].values.reshape(-1,1))
# Признак утреннего входа (до 12)
sessions['morning'] = first_time.apply(lambda x: 1 if x.hour < 12 else 0)
# Значение часа первого входа
sessions['first_hour'] = first_time.apply(lambda x: x.hour)
sessions['first_hour_scaled'] = scaler.fit_transform(sessions['first_hour'].values.reshape(-1,1))

In [21]:
sessions.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site9,time9,site10,time10,target,year_month,year_month_scaled,morning,first_hour,first_hour_scaled
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,0,NaT,0,NaT,0,NaT,0,NaT,...,0,NaT,0,NaT,0,201402,0.634518,1,10,-0.724338
2,890,2014-02-22 11:19:50,941,2014-02-22 11:19:50,3847,2014-02-22 11:19:51,941,2014-02-22 11:19:51,942,2014-02-22 11:19:51,...,1516,2014-02-22 11:20:15,1518,2014-02-22 11:20:16,0,201402,0.634518,1,11,-0.407823
3,14769,2013-12-16 16:40:17,39,2013-12-16 16:40:18,14768,2013-12-16 16:40:19,14769,2013-12-16 16:40:19,37,2013-12-16 16:40:19,...,14768,2013-12-16 16:40:22,14768,2013-12-16 16:40:24,0,201312,-1.485314,0,16,1.174749
4,782,2014-03-28 10:52:12,782,2014-03-28 10:52:42,782,2014-03-28 10:53:12,782,2014-03-28 10:53:42,782,2014-03-28 10:54:12,...,782,2014-03-28 10:56:12,782,2014-03-28 10:56:42,0,201403,0.658072,1,10,-0.724338
5,22,2014-02-28 10:53:05,177,2014-02-28 10:55:22,175,2014-02-28 10:55:22,178,2014-02-28 10:55:23,177,2014-02-28 10:55:23,...,177,2014-02-28 10:57:06,178,2014-02-28 10:57:11,0,201402,0.634518,1,10,-0.724338


In [22]:
from scipy.sparse import hstack

In [23]:
X = hstack([sites_sparse, sessions['morning'].values.reshape(-1,1)])
get_roc(X, y)

0.598954195195806

In [24]:
X = hstack([X, sessions['year_month_scaled'].values.reshape(-1,1)])
get_roc(X, y)

0.6018574717115714

🛠 Добавить новые параметры. Добиться улучшения модели. 